# Partícionálás

In [ ]:
import cv2
import numpy as np
from pdf2image import convert_from_path

def load_images_from_pdf(pdf_path, size=None):
    """
    Load OpenCV images from a PDF file.
    :param pdf_path: path of the PDF file
    :param size: the preferred size in pixels as a (width, height) tuple
    :return: list of OpenCV images
    """
    pil_images = convert_from_path(pdf_path, size=size)
    images = [
        cv2.cvtColor(np.array(pil_image), cv2.COLOR_BGR2GRAY)
        for pil_image in pil_images
    ]
    return images

In [ ]:
images = load_images_from_pdf('samples/test12.pdf', size=(2500, None))

In [ ]:
image = images[1]
image.shape

In [ ]:
from matplotlib import pyplot as plt

plt.figure(figsize=(5, 10))
plt.imshow(image, cmap='gray')
plt.show()

In [ ]:
def find_segments(values, background_color):
    """
    Find the segments with non-background colors in the iterable.
    :param values: intensity values
    :param background_color: the background color which should be skipped
    :return: list of segments as [start, end) tuples of indices
    """
    segments = []
    start = None
    end = None
    for i, value in enumerate(values):
        if value != background_color:
            if start is None:
                start = i
        elif start is not None:
            end = i
            segments.append((start, end))
            start = None
    return segments

In [ ]:
def find_max_split_position(segments):
    """
    Calculate the spacing between the segments.
    :param segments: list of segments as [start, end) intervals
    :return: tuple of split position with maximal spacing and the spacing itself
    """
    spacing = []
    max_space = 0
    max_split_position = None
    for i in range(len(segments) - 1):
        space = segments[i + 1][0] - segments[i][1]
        if space > max_space:
            max_space = space
            max_split_position = (segments[i][1] + segments[i + 1][0]) // 2
    return max_split_position, max_space

In [ ]:
row_profile = np.mean(image, axis=1)
segments = find_segments(row_profile, 255)
find_max_split_position(segments)

In [ ]:
def find_split_position(image, region):
    """
    Find the optimal position of the splitting.
    :param image: a NumPy image
    :param region: the considered part of the image
    :return: tuple of axis (0 or 1) and position
    """
    roi = image[
        region.row:region.row + region.n_rows,
        region.column:region.column + region.n_columns]
    row_profile = np.mean(roi, axis=1)
    segments = find_segments(row_profile, 255)
    row_split, row_space = find_max_split_position(segments)
    if row_split is not None:
        row_split += region.row
    column_profile = np.mean(roi, axis=0)
    segments = find_segments(column_profile, 255)
    column_split, column_space = find_max_split_position(segments)
    if column_split is not None:
        column_split += region.column
    if column_split is None:
        return 0, row_split
    if row_split is None:
        return 1, column_split
    if row_space >= column_space:
        return 0, row_split
    else:
        return 1, column_split

In [ ]:
from region import Region

region = Region(0, 0, image.shape[0], image.shape[1])
axis, position = find_split_position(image, region)

In [ ]:
def split_region(region, axis, position):
    """
    Split the region into two half by the given axis.
    :param region: region of interest
    :param axis: 0 or 1 for row and column
    :param position: position of the splitting
    :return: tuple of two region objects
    """
    if axis == 0:
        upper_region = Region(
            region.row, region.column,
            position - region.row, region.n_columns
        )
        lower_region = Region(
            position, region.column,
            region.row + region.n_rows - position, region.n_columns
        )
        return upper_region, lower_region
    else:
        left_region = Region(
            region.row, region.column,
            region.n_rows, position - region.column
        )
        right_region = Region(
            region.row, position,
            region.n_rows, region.column + region.n_columns - position
        )
        return left_region, right_region

In [ ]:
region_1, region_2 = split_region(region, axis, position)
print(region)
print(region_1)
print(region_2)

In [ ]:
def build_tree(image, region):
    """
    Building a binary tree by splitting the image horizontally or vertically.
    :param image: the image which should be recursively partitioned
    :param region: the considered part of the image
    :return: a binary tree of regions
    """
    axis, position = find_split_position(image, region)
    if position is not None:
        region_1, region_2 = split_region(region, axis, position)
        left_child = build_tree(image, region_1)
        right_child = build_tree(image, region_2)
        return [left_child, right_child]
    else:
        return region

In [ ]:
region = Region(0, 0, image.shape[0], image.shape[1])
tree = build_tree(image, region)

In [ ]:
tree

## Display the resulted partitioning

In [ ]:
result = cv2.merge((image, image, image))

In [ ]:
def draw_regions(tree):
    global result
    if isinstance(tree, list):
        draw_regions(tree[0])
        draw_regions(tree[1])
    else:
        region = tree
        x_1 = region.column
        y_1 = region.row
        x_2 = x_1 + region.n_columns
        y_2 = y_1 + region.n_rows
        result = cv2.rectangle(result, (x_1, y_1), (x_2, y_2), (255, 0, 0))

In [ ]:
draw_regions(tree)

In [ ]:
cv2.imwrite('/tmp/splitted.png', result)